In [0]:
# Import Modules
from pyspark.sql.functions import current_timestamp

In [0]:
# Add Ingestion Date
def add_ingestion_date(input_df):
    output_df = input_df.withColumn('ingestion_date', current_timestamp())

    return output_df